<h1 style="color: #1e3a8a; background-color: #dbeafe; padding: 12px; border-left: 5px solid #2563eb; margin: 20px 0; font-weight: bold;">Test de Rechargement du Modèle et Prédiction</h1>

**Objectif** : Vérifier que tous les artefacts peuvent être rechargés correctement et que le modèle fonctionne.

**Tests à effectuer** :
1. Rechargement de tous les artefacts (modèle, encoders, features, métriques)
2. Vérification de la cohérence des données
3. Test de prédiction sur un échantillon
4. Validation du format de sortie

In [1]:
# Imports
import pandas as pd
import numpy as np
import pickle
import json
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("Imports terminés")

Imports terminés


<h2 style="color: #1e40af; background-color: #eff6ff; padding: 10px; border-left: 4px solid #3b82f6; margin: 15px 0;">Rechargement des Artefacts</h2>

In [2]:
print("="*80)
print("RECHARGEMENT DES ARTEFACTS")
print("="*80)
print()

models_dir = Path('../models')

# 1. Charger le modèle
print("1. Chargement du modèle LightGBM...")
with open(models_dir / 'model.pkl', 'rb') as f:
    model = pickle.load(f)
print(f"   Type: {type(model).__name__}")
print(f"   Nombre d'estimateurs: {model.n_estimators}")
print()

# 2. Charger les encoders
print("2. Chargement des Label Encoders...")
with open(models_dir / 'label_encoders.pkl', 'rb') as f:
    label_encoders = pickle.load(f)
print(f"   Nombre d'encodeurs: {len(label_encoders)}")
print(f"   Variables: {list(label_encoders.keys())}")
print()

print("3. Chargement des One-Hot Encoders...")
with open(models_dir / 'onehot_encoder.pkl', 'rb') as f:
    onehot_encoders = pickle.load(f)
print(f"   Nombre d'encodeurs: {len(onehot_encoders)}")
print()

# 3. Charger les noms de features
print("4. Chargement des noms de features...")
with open(models_dir / 'feature_names.pkl', 'rb') as f:
    feature_names = pickle.load(f)
print(f"   Nombre de features: {len(feature_names)}")
print(f"   Premières features: {feature_names[:5]}")
print()

# 4. Charger les métriques
print("5. Chargement des métriques...")
with open(models_dir / 'metrics.json', 'r') as f:
    metrics = json.load(f)
print(f"   AUC-ROC: {metrics['auc_roc']:.4f}")
print(f"   Recall: {metrics['recall']:.4f}")
print()

# 5. Charger le seuil optimal
print("6. Chargement du seuil optimal...")
with open(models_dir / 'threshold.json', 'r') as f:
    threshold_data = json.load(f)
optimal_threshold = threshold_data['optimal_threshold']
print(f"   Seuil optimal: {optimal_threshold:.4f}")
print()

print("Tous les artefacts ont été chargés avec succès")
print()

RECHARGEMENT DES ARTEFACTS

1. Chargement du modèle LightGBM...
   Type: LGBMClassifier
   Nombre d'estimateurs: 300

2. Chargement des Label Encoders...
   Nombre d'encodeurs: 5
   Variables: ['NAME_CONTRACT_TYPE', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'EMERGENCYSTATE_MODE', 'PREV_FLAG_LAST_APPL_PER_CONTRACT_<LAMBDA_0>']

3. Chargement des One-Hot Encoders...
   Nombre d'encodeurs: 32

4. Chargement des noms de features...
   Nombre de features: 911
   Premières features: ['NAME_CONTRACT_TYPE', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL']

5. Chargement des métriques...
   AUC-ROC: 0.7826
   Recall: 0.6628

6. Chargement du seuil optimal...
   Seuil optimal: 0.5225

Tous les artefacts ont été chargés avec succès



<h2 style="color: #1e40af; background-color: #eff6ff; padding: 10px; border-left: 4px solid #3b82f6; margin: 15px 0;">Test de Prédiction</h2>

In [ ]:
print("="*80)
print("TEST DE PRÉDICTION")
print("="*80)
print()

# Charger les données de test
print("Chargement d'un échantillon de données...")
data_path = Path('../data/app_train_models.csv')
df_sample = pd.read_csv(data_path, nrows=5)

# Séparer features et target
X_sample = df_sample.drop(['TARGET', 'SK_ID_CURR'], axis=1)
y_sample = df_sample['TARGET']

print(f"Échantillon chargé: {X_sample.shape}")
print()

# Appliquer le preprocessing (comme dans le notebook de production)
print("Application du preprocessing...")
X_encoded = X_sample.copy()

# Label Encoding
for col, encoder in label_encoders.items():
    if col in X_encoded.columns:
        # Gérer les catégories inconnues
        unknown_mask = ~X_encoded[col].isin(encoder.classes_)
        if unknown_mask.sum() > 0:
            X_encoded.loc[unknown_mask, col] = encoder.classes_[0]
        X_encoded[col] = encoder.transform(X_encoded[col])

# One-Hot Encoding
for col, encoder in onehot_encoders.items():
    if col in X_encoded.columns:
        encoded_data = encoder.transform(X_encoded[[col]])
        feature_names_temp = [f"{col}_{cat}" for cat in encoder.categories_[0]]
        encoded_df = pd.DataFrame(encoded_data, columns=feature_names_temp, index=X_encoded.index)
        X_encoded = X_encoded.drop(columns=[col])
        X_encoded = pd.concat([X_encoded, encoded_df], axis=1)

# Nettoyer les noms de colonnes
X_encoded.columns = X_encoded.columns.str.replace('[^A-Za-z0-9_]', '_', regex=True)

print(f"Données encodées: {X_encoded.shape}")
print()

# Vérifier que les colonnes correspondent
if list(X_encoded.columns) == feature_names:
    print("Les noms de features correspondent")
else:
    print("ATTENTION: Les noms de features ne correspondent pas")
    missing = set(feature_names) - set(X_encoded.columns)
    extra = set(X_encoded.columns) - set(feature_names)
    if missing:
        print(f"Features manquantes: {len(missing)}")
    if extra:
        print(f"Features en trop: {len(extra)}")
print()

# Faire les prédictions
print("Génération des prédictions...")
y_proba = model.predict_proba(X_encoded)[:, 1]
y_pred_default = (y_proba >= 0.5).astype(int)
y_pred_optimal = (y_proba >= optimal_threshold).astype(int)

print("Prédictions générées")
print()

# Afficher les résultats
print("RÉSULTATS DES PRÉDICTIONS:")
print("-" * 80)
results_df = pd.DataFrame({
    'Client_ID': range(len(y_sample)),
    'Vrai_Label': y_sample.values,
    'Probabilité': y_proba,
    'Prédiction_0.5': y_pred_default,
    'Prédiction_Optimal': y_pred_optimal
})

print(results_df.to_string(index=False))
print()

# Niveau de risque
def get_risk_level(proba):
    if proba < 0.3:
        return 'FAIBLE'
    elif proba < 0.6:
        return 'MOYEN'
    else:
        return 'ÉLEVÉ'

print("ANALYSE DU RISQUE:")
print("-" * 80)
for i in range(len(y_sample)):
    risk = get_risk_level(y_proba[i])
    decision = 'REFUSÉ' if y_pred_optimal[i] == 1 else 'ACCEPTÉ'
    print(f"Client {i}: Probabilité={y_proba[i]:.4f} | Risque={risk} | Décision={decision}")
print()

print("Test de prédiction terminé avec succès")
print()

TEST DE PRÉDICTION

Chargement d'un échantillon de données...
Échantillon chargé: (5, 645)

Application du preprocessing...
Données encodées: (5, 912)

ATTENTION: Les noms de features ne correspondent pas
Features en trop: 1

Génération des prédictions...


[LightGBM] [Fatal] The number of features in data (912) is not the same as it was in training data (911).
You can set ``predict_disable_shape_check=true`` to discard this error, but please be aware what you are doing.


LightGBMError: The number of features in data (912) is not the same as it was in training data (911).
You can set ``predict_disable_shape_check=true`` to discard this error, but please be aware what you are doing.

<h2 style="color: #1e40af; background-color: #eff6ff; padding: 10px; border-left: 4px solid #3b82f6; margin: 15px 0;">Format de Sortie pour l'API</h2>

In [ ]:
print("="*80)
print("FORMAT DE SORTIE POUR L'API")
print("="*80)
print()

# Créer le format de réponse pour l'API
from datetime import datetime

print("Exemple de réponse JSON pour l'API:")
print()

# Prendre le premier client comme exemple
client_id = 100001  # ID fictif
proba = float(y_proba[0])
prediction = int(y_pred_optimal[0])
risk = get_risk_level(proba)

api_response = {
    "client_id": client_id,
    "probability_default": round(proba, 4),
    "prediction": prediction,
    "decision": "REFUSE" if prediction == 1 else "APPROVE",
    "risk_level": risk.lower(),
    "threshold_used": round(optimal_threshold, 4),
    "model_version": "1.0",
    "timestamp": datetime.now().isoformat()
}

import json
print(json.dumps(api_response, indent=2))
print()

print("Format validé pour l'API")
print()

FORMAT DE SORTIE POUR L'API

Exemple de réponse JSON pour l'API:

{
  "client_id": 100001,
  "probability_default": 0.8742,
  "prediction": 1,
  "decision": "REFUSE",
  "risk_level": "\u00e9lev\u00e9",
  "threshold_used": 0.4955,
  "model_version": "1.0",
  "timestamp": "2025-12-01T11:52:13.994930"
}

Format validé pour l'API



<h2 style="color: #1e40af; background-color: #eff6ff; padding: 10px; border-left: 4px solid #3b82f6; margin: 15px 0;">Résumé des Tests</h2>

In [ ]:
print("="*80)
print("RÉSUMÉ DES TESTS")
print("="*80)
print()

print("ARTEFACTS TESTÉS:")
print(f"  1. Modèle LightGBM               : OK")
print(f"  2. Label Encoders ({len(label_encoders)})           : OK")
print(f"  3. One-Hot Encoders ({len(onehot_encoders)})        : OK")
print(f"  4. Feature Names ({len(feature_names)})          : OK")
print(f"  5. Métriques de référence        : OK")
print(f"  6. Seuil optimal ({optimal_threshold:.4f})      : OK")
print()

print("PRÉDICTIONS TESTÉES:")
print(f"  Nombre d'échantillons testés     : {len(y_sample)}")
print(f"  Prédictions réussies             : {len(y_sample)}")
print(f"  Format de sortie validé          : OK")
print()

print("="*80)
print("TOUS LES TESTS SONT PASSÉS AVEC SUCCÈS")
print("="*80)
print()

print("Le modèle est prêt à être intégré dans l'API FastAPI")
print()

RÉSUMÉ DES TESTS

ARTEFACTS TESTÉS:
  1. Modèle LightGBM               : OK
  2. Label Encoders (5)           : OK
  3. One-Hot Encoders (32)        : OK
  4. Feature Names (912)          : OK
  5. Métriques de référence        : OK
  6. Seuil optimal (0.4955)      : OK

PRÉDICTIONS TESTÉES:
  Nombre d'échantillons testés     : 5
  Prédictions réussies             : 5
  Format de sortie validé          : OK

TOUS LES TESTS SONT PASSÉS AVEC SUCCÈS

Le modèle est prêt à être intégré dans l'API FastAPI

